# Data preprocessing for Textmining

____

금일 처리해야하는 사항들 

* feed 내에서 나온 comment들을 모아야 합니다.
> output: [댓글, [대댓글,대댓글,대댓글]] 의 형태로 나타나게끔 묶어줍니다.

In [127]:
# 필요한 package 
import pandas as pd 
import ast #str to list
from tqdm import tqdm 
import tqdm
import numpy as np
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)
import re
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

In [196]:

def read_data():
    hyatt= pd.read_csv("hyatt_comment",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_comment",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_comment",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham_comment",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()
test=hilton[:30]

In [197]:
# comment
def string_to_list(df,columns):
    for col in columns:
        df[col]=df[col].apply(lambda x: ast.literal_eval(x))
        
for hotel in (hilton,hyatt,holiday,wyndham):
    string_to_list(hotel,["comment","comment_like","comment_writer","Recomment"])

print(hilton['comment'][0][0])
hilton.head()# 애초에 list

❤️❤️❤️


,post_id,comment,comment_like,comment_writer,Recomment
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[[@allpurple10 💙🍪, <Profile hilton (428491337..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[[@allpurple10 thank you!, <Profile hilton (4..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [[@erattray1 I saw that earli..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [[@patriciatorressl Waldorf?, <Profile gi..."


comment 첫번째에 대한 답글 -> Recomment0에서 0에 있는 것들 
comment 두번째에 대한 답글 -> Recomment0에서 1에 있는 것들 
comment 세번째에 대한 답글 -> Recomment0에서 2에 있는 것들 

document=[comment,[recomment],]

In [198]:
def string_extract(x):
    # input : 3차 
    new_list=[]
    for i in x:
        if len(i)==0:
            new_list.append([])
            
        elif len(i)==1:
            new_list.append([i[0][0]])
        
        else : 
            string_lst=[]
            for j in range(len(i)-1):
                string_lst.append(i[j][0])
            new_list.append(string_lst)
    
    return new_list

In [199]:
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["new_recom"]=hotel["Recomment"].apply(lambda x: string_extract(x))
    hotel.drop(["Recomment"],axis=1,inplace=True)

hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ..."


new_recomment에 대한 정리는 끝    
-> 이제 comment와 new_recomment를 묶어보자. numpy가 아니기에 지금 굳이 묶을 필요는 x) ->묶으려면 아예 위계가 없게 묶는게 나은데 그건 추후에 하겠습니다.

## 가장 simple한 nltk 사용

우선 가장 먼저해야 하는 것은 text_cleaning 이지만 social 데이터의 경우에는 바로 text_cleaning을 하지 않는 것이 좋다. 이 연구에서는 그래서 text_cleaning을 하지 않고 진행하도록 하겠습니다. 우선 영어 코퍼스의 토큰화를 위한 도구를 NLTK를 통해서 진행하겠습니다.

In [205]:
# test=hilton[:30]
test.head()

,post_id,comment,comment_like,comment_writer,new_recom
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ..."


In [35]:
from nltk.tokenize import word_tokenize
import nltk 
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunkeun_jo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [126]:
# comment에 대한 tokenize
def cleaned_text(text):
    # 누구를 tag한 태그 표시는 지우기
    #전의 text
    prev_text=text
    while True:
        text=re.sub("@[\d\w]+","",text)
        
        if prev_text!=text:
            prev_text=text
        else:
            break
            
    return text

def text_tokenize(text):
    return [word_tokenize(cleaned_text(sentence)) for sentence in text]

for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["comment_token"]=hotel["comment"].apply(lambda x: text_tokenize(x))
    
hilton.head()

# comment에 대한 토큰화 작업 끝 

,post_id,comment,comment_like,comment_writer,new_recom,commoent_token,comment_token
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[[❤️❤️❤️], [👏], [🤤❤️], [OMG, WHAT, THIS, IS, A...","[[❤️❤️❤️], [👏], [🤤❤️], [OMG, WHAT, THIS, IS, A..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[[This, is, awesome, !, !, !, 👏👏👏], [Thank, yo...","[[This, is, awesome, !, !, !, 👏👏👏], [Thank, yo..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],"[[., ., ., ., #, staugustine, #, staugustinefl...","[[., ., ., ., #, staugustine, #, staugustinefl..."
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[[., ., ., ., #, boca, #, bocaraton, #, resort...","[[., ., ., ., #, boca, #, bocaraton, #, resort..."


comment와 recomment들을 하나로 묶어서 한 comment의 집단이 어떤 sentiment를 가지고 있는지 파악한다.

In [280]:
# recomment를 회전시키는 함수 
def recom_to_sentence(_list):
    sentence_lst=[]
    for i in _list:
        sentence=""
        for rec in i:
            sentence= sentence+" "+rec
        sentence_lst.append(sentence)
    return sentence_lst
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["recomment"]=hotel["new_recom"].apply(lambda x: recom_to_sentence(x))
    
hilton.head()


,post_id,comment,comment_like,comment_writer,new_recom,recomment
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[ @allpurple10 💙🍪, @hamptonrockyhill we're pr..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[ @allpurple10 thank you!, , , , , , , , , , ,..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[, , , , @erattray1 I saw that earlier. Let’s..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],[]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[, @patriciatorressl Waldorf? @giles.ynwa si!..."


In [281]:
test

,post_id,comment,comment_like,comment_writer,new_recom,recomment,final
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[ @allpurple10 💙🍪, @hamptonrockyhill we're pr...","[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[ @allpurple10 thank you!, , , , , , , , , , ,...","[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[, , , , @erattray1 I saw that earlier. Let’s...","[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],[],[.\n.\n.\n.\n#staugustine #staugustinefl #stau...
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[, @patriciatorressl Waldorf? @giles.ynwa si!...",[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...
5,6,[.\n.\n.\n.\n#waikoloavillage #bigisland #hawa...,[0],[4284913372],[[]],[],[.\n.\n.\n.\n#waikoloavillage #bigisland #hawa...
6,7,[.\n.\n.\n.\n#bali #jakarta #indonesia #balii...,"[0, 1]","[4284913372, 1551711226]","[[], []]","[, ]",[.\n.\n.\n.\n#bali #jakarta #indonesia #balii...
7,8,[.\n.\n.\n.\n#indianwells #palmsprings #southe...,"[0, 0]","[4284913372, 27981322088]","[[], []]","[, ]",[.\n.\n.\n.\n#indianwells #palmsprings #southe...
8,9,[.\n.\n.\n.\n#prague #praha #czechrepublic #cz...,"[0, 1, 0]","[4284913372, 15942259, 1938464885]","[[], [], []]","[, , ]",[.\n.\n.\n.\n#prague #praha #czechrepublic #cz...
9,10,[.\n.\n.\n.\n\n#paris #france #parisjetaime #p...,[0],[4284913372],[[]],[],[.\n.\n.\n.\n\n#paris #france #parisjetaime #p...


In [285]:

    
sentence_lst=[]
for i in range(len(test["comment"][0])):
    sentence=""
    sentence=test["comment"][0][i]+" "+test["recomment"][0][i]
    sentence_lst.append(sentence)
print(sentence_lst)
print(len(sentence_lst))

['❤️❤️❤️  @allpurple10 💙🍪', "👏  @hamptonrockyhill we're pretty excited about it!", '🤤❤️  @erry_sky_cap enjoy! We hope you can try it at home 😍', 'OMG WHAT THIS IS AWESOME  @patcondy \U0001f92f\U0001f92f\U0001f92f', 'I miss staying at Hilton hotels so much! So my dad made these for me this morning haha ', 'This is the best news!!! Can’t wait to bake these at home 🍪 ❤️THANK YOU ', "❤️❤️❤️ super team thank you @hilton  @socialslothcafe you're very welcome!", "@marina_bisdorf I love these cookies. We must make them when this is over.  @oliviacerre yes please!!! @oliviacerre don't forget to tag us! We can't wait to see how they turn out.", '@tlspark @rachel_kolisi @jcsmith7 @sarah_jodi12  @thishan3 🤤🤤', '@allisonrosnellhayes IMPORTANT  @nlentine God I miss those cookies!', '@noyona16 remember the yummy cookie?? 🍪🍪  @sany11_1 yup', '@halachmaanya ', '@danguarino ', '@anisamirah ', '@pattib558 ', '@nglett88 ']
16


In [296]:
a=np.array(test["comment"][0]).reshape(-1,1)
b=np.array(test["recomment"][0]).reshape(-1,1)
test["final"][0]=np.hstack((a,b))

C:\Users\sunkeun_jo\anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sunkeun_jo\anaconda3\envs\textmining\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [305]:
for i in test["final"][0]:
    print(i[0]+i[1])

❤️❤️❤️ @allpurple10 💙🍪
👏 @hamptonrockyhill we're pretty excited about it!
🤤❤️ @erry_sky_cap enjoy! We hope you can try it at home 😍
OMG WHAT THIS IS AWESOME @patcondy 🤯🤯🤯
I miss staying at Hilton hotels so much! So my dad made these for me this morning haha
This is the best news!!! Can’t wait to bake these at home 🍪 ❤️THANK YOU
❤️❤️❤️ super team thank you @hilton @socialslothcafe you're very welcome!
@marina_bisdorf I love these cookies. We must make them when this is over. @oliviacerre yes please!!! @oliviacerre don't forget to tag us! We can't wait to see how they turn out.
@tlspark @rachel_kolisi @jcsmith7 @sarah_jodi12 @thishan3 🤤🤤
@allisonrosnellhayes IMPORTANT @nlentine God I miss those cookies!
@noyona16 remember the yummy cookie?? 🍪🍪 @sany11_1 yup
@halachmaanya
@danguarino
@anisamirah
@pattib558
@nglett88
